<font size=6.5 color="blue">** Recommendations with pandas **</font>

# 相似度计算

In [25]:
import pandas as pd
import numpy as np

In [2]:
critics={
    'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,'Just My Luck': 3.0, 'Superman Returns': 3.5, 
                  'You, Me and Dupree': 2.5, 'The Night Listener': 3.0},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
                     'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
    'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,'Superman Returns': 3.5, 'The Night Listener': 4.0},
    'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5},
    'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
                     'You, Me and Dupree': 2.0}, 
    'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,'The Night Listener': 3.0, 'Superman Returns': 5.0, 
                      'You, Me and Dupree': 3.5},
    'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}} 

In [3]:
critics = pd.DataFrame(critics)

In [4]:
critics

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Michael Phillips,Mick LaSalle,Toby
Just My Luck,3.0,1.5,NaN,3.0,NaN,2.0,NaN
Lady in the Water,NaN,3.0,3.0,2.5,2.5,3.0,NaN
Snakes on a Plane,3.5,3.5,4.0,3.5,3.0,4.0,4.5
Superman Returns,4.0,5.0,5.0,3.5,3.5,3.0,4.0
The Night Listener,4.5,3.0,3.0,3.0,4.0,3.0,NaN
"You, Me and Dupree",2.5,3.5,3.5,2.5,NaN,2.0,1.0


In [6]:
from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean

In [18]:
def sim_pearson(prefs,p1,p2):
    '''输入一个dataFrame，及任意两列的名字'''
    df1=prefs[[p1,p2]]
    df2=df1[(df1[p1]>=0) & (df1[p2]>=0)]
    
    r,p_value=pearsonr(df2.iloc[:,0],df2.iloc[:,1])
    
    if p_value==1: return 0
    return r

def sim_distance(prefs,p1,p2):
    df1=prefs[[p1,p2]]
    df2=df1[(df1[p1]>=0) & (df1[p2]>=0)]
    d=euclidean(df2.iloc[:,0],df2.iloc[:,1])
    return 1/(1+d)

In [17]:
def topMatches(prefs,p1,n=5,similarity=sim_pearson):
    '''返回一个：p1与其他元素的相关度的DataFrame'''
    
    scores=[(other, similarity(prefs,p1, other)) for other in prefs.columns if other !=p1 ]
    scores=pd.DataFrame(scores, columns=['Name', 'Score'])
    scores=scores.sort_values(by='Score', ascending=False)
    return scores[:n]

In [15]:
sim_distance(critics,'Lisa Rose', 'Gene Seymour')

0.29429805508554946

# 推荐物品
- 输入“人”
- 输出对未看过电影的评分预测排名

In [ ]:
def getRecommendattions(prefs,p1,similarity=sim_pearson):
    df1=prefs[p1][np.isnan(prefs[p1])]
    

In [41]:
r=topMatches(critics,'Toby',n=len(critics.columns)-1)
r

,Name,Score
3,Lisa Rose,0.991241
5,Mick LaSalle,0.924473
0,Claudia Puig,0.893405
2,Jack Matthews,0.662849
1,Gene Seymour,0.381246
4,Michael Phillips,-1.000000


In [40]:
critics['Toby']

Just My Luck          NaN
Lady in the Water     NaN
Snakes on a Plane     4.5
Superman Returns      4.0
The Night Listener    NaN
You, Me and Dupree    1.0
Name: Toby, dtype: float64

In [71]:
rr=r.T.iloc[1:,:].rename(columns=r.iloc[:,0]).reindex(columns=critics.columns)
rr

,Claudia Puig,Gene Seymour,Jack Matthews,Lisa Rose,Michael Phillips,Mick LaSalle,Toby
Score,0.893405,0.381246,0.662849,0.991241,-1,0.924473,NaN


In [70]:
np.dot(rr,critics)

array([[nan, 8.345183611007972, nan, 9.375534475390111, nan,
        9.461808577589977, nan]], dtype=object)